In [13]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from src.dataloaders_and_sets.simple_dataset import SimpleDataset
from tqdm import tqdm
from torch.utils.data import DataLoader
import torch.optim as optim
from torcheval.metrics.functional import r2_score
from torch.optim.lr_scheduler import ExponentialLR
import tensorboard
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Loading Data and Preprocessing

In [2]:
data_with_targets = pd.read_csv('data/data.csv', index_col=0)

In [3]:
data = data_with_targets.fillna(0.0)

In [54]:
# remove the possible y labels:
y_labels = ['primary_disease', 'gender', 'age', 'dataset']
data_columns = [col for col in data.columns if col not in y_labels]
y = "gender"
# TODO: makes no sense when using a autoencoder...
X_train, X_test, y_train, y_test = train_test_split(data[data_columns], data[y_labels], train_size=0.8, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.8, random_state=42)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(7539, 17137) (7539, 4) (1885, 17137) (1885, 4) (2356, 17137) (2356, 4)


In [5]:
# transform dataset for the simple autoencoder

transform_fc_ae = {
    "z_score": "per_sample",
    "most_variant": 5000,
}

In [6]:
fc_ae_dataset = SimpleDataset(X_train, transform=transform_fc_ae)
fc_ae_dataset[0]


/home/fes/Nextcloud/Uni/B.Sc. Bioinfo/Bachelorarbeit/thesis/src/dataloaders_and_sets/simple_dataset.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.tensor(self.data.iloc[index], dtype=torch.float32)


tensor([-0.9623, -0.0346,  0.1806,  ..., -0.5805,  0.5189,  0.0858])

In [7]:
# config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("{} is used".format(device))

cuda is used


# Setup Tensorboard

In [8]:
writer = SummaryWriter("logs/autoencoder")


In [9]:
input_dim = len(fc_ae_dataset[0])
hidden_one_dim = 512
hidden_two_dim = 128
z_dim = 64

num_epochs = 50
batch_size = 512
learning_rate = 3e-3
beta = 0.1
print(input_dim)

5000


run `tensorboard --logdir=./`

# Training

In [80]:
# dataset loading
from src.models.fc import VAE

train_dataset = SimpleDataset(X_train, transform=transform_fc_ae)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = SimpleDataset(X_val, transform=transform_fc_ae)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
model = VAE(input_size=input_dim, hidden_one_size=hidden_one_dim, hidden_two_size=hidden_two_dim, z_size=z_dim).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ExponentialLR(optimizer, gamma=0.95)

def loss_mse(truth, pred):
    return ((truth - pred)**2).sum()
def loss_kl(mu, sigma):
    return (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()

In [17]:
model.eval()
writer.add_graph(model, train_dataset[0].to(device))
writer.close()

/home/fes/Nextcloud/Uni/B.Sc. Bioinfo/Bachelorarbeit/thesis/src/dataloaders_and_sets/simple_dataset.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.tensor(self.data.iloc[index], dtype=torch.float32)
/home/fes/anaconda3/envs/Helmholtz/lib/python3.11/site-packages/torch/jit/_trace.py:1116: TracerWarning: Trace had nondeterministic nodes. Did you forget call .eval() on your model? Nodes:
	%60 : Float(64, strides=[1], requires_grad=0, device=cuda:0) = aten::normal(%53, %58, %59) # /home/fes/anaconda3/envs/Helmholtz/lib/python3.11/site-packages/torch/distributions/normal.py:70:0
This may cause errors in trace checking. To disable trace checking, pass check_trace=False to torch.jit.trace()
  _check_trace(
/home/fes/anaconda3/envs/Helmholtz/lib/python3.11/site-packages/torch/jit/_trace.py:11

In [13]:
for epoch in range(num_epochs):
    train_loss = 0.0
    val_loss = 0.0
    number_train_batches = 0
    number_val_batches = 0
    
    # Training loop
    model.train()
    for batch_idx, batch in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch+1} Training")):
        batch = batch.to(device)
        optimizer.zero_grad()
        recon_batch, mu, sigma = model(batch)

        # Compute training loss
        single_loss_mse = loss_mse(batch, recon_batch)
        single_loss_kl = loss_kl(mu, sigma)
        # print("MSE: {}, KL: {}".format(single_loss_mse, single_loss_kl))
        loss = single_loss_mse + beta * single_loss_kl
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
        number_train_batches += 1

    scheduler.step()


    # Validation loop
    model.eval()
    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch+1} Validation")):
            batch = batch.to(device)
            reconstructed, _, _ = model(batch)
            loss = r2_score(reconstructed, batch)
            #print(loss.item())
            #print(reconstructed - batch)
            val_loss += loss.item()
            

    # Calculate average losses
    avg_train_loss = train_loss / len(train_dataloader.dataset)
    avg_val_loss = val_loss / len(train_dataloader.dataset)

    # Output average losses for the epoch
    print(f"Epoch {epoch+1}:")
    print(f"\tTraining Loss: {avg_train_loss:.6f}")
    print(f"\tValidation Loss: {avg_val_loss:.6f}")


Epoch 1 Training:   0%|          | 0/15 [00:00<?, ?it/s]/home/fes/Nextcloud/Uni/B.Sc. Bioinfo/Bachelorarbeit/thesis/src/dataloaders_and_sets/simple_dataset.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.tensor(self.data.iloc[index], dtype=torch.float32)
Epoch 1 Validation: 100%|██████████| 15/15 [00:05<00:00,  2.64it/s]


Epoch 1:
	Training Loss: nan
	Validation Loss: nan


Epoch 2 Training:  40%|████      | 6/15 [00:02<00:04,  2.23it/s]


KeyboardInterrupt: 

## Simple autoencoder

In [52]:
from src.models.fc import AE

batch_size = 512
learning_rate = 6e-4
num_epochs = 200

hidden_one_dim = 512
hidden_two_dim = 128
z_dim = 64


ae_dataset = SimpleDataset(X_train, transform=transform_fc_ae)
ae_dataloader = DataLoader(ae_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
ae = AE(input_size=input_dim, hidden_one_size=hidden_one_dim, hidden_two_size=hidden_two_dim, z_size=z_dim).to(device)
optimizer = optim.Adam(ae.parameters(), lr=learning_rate)
scheduler = ExponentialLR(optimizer, gamma=0.992)
loss_mse = nn.MSELoss()

ae_val_dataset = SimpleDataset(X_val, transform=transform_fc_ae)
ae_val_dataloader = DataLoader(ae_val_dataset, batch_size=batch_size, shuffle=True)


In [55]:
ae.eval()
writer.add_graph(ae, ae_dataset[0].to(device))
writer.close()


/home/fes/Nextcloud/Uni/B.Sc. Bioinfo/Bachelorarbeit/thesis/src/dataloaders_and_sets/simple_dataset.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.tensor(self.data.iloc[index], dtype=torch.float32)


In [33]:
counter = 0
for epoch in range(10):
    for iter, batch in enumerate(ae_dataloader):
        print(f"counter: {counter}, batch: {batch}")
        counter += 1 
        break

/home/fes/Nextcloud/Uni/B.Sc. Bioinfo/Bachelorarbeit/thesis/src/dataloaders_and_sets/simple_dataset.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.tensor(self.data.iloc[index], dtype=torch.float32)


counter: 0, batch: tensor([[-0.9623, -0.0346,  0.1806,  ..., -0.5805,  0.5189,  0.0858],
        [-1.1006,  0.6535,  0.3943,  ..., -1.6190, -1.2263,  0.0644],
        [-0.9457, -0.8053, -0.7376,  ..., -1.4371, -1.7079, -0.5195],
        ...,
        [ 0.0326,  0.0624, -0.9356,  ..., -1.6549, -1.0750,  0.1570],
        [-0.9285, -0.6192, -1.2086,  ..., -1.6032, -1.6032, -0.2125],
        [-1.1774,  2.9026,  2.7290,  ..., -0.7105, -1.5929, -0.1824]])
counter: 1, batch: tensor([[-0.9623, -0.0346,  0.1806,  ..., -0.5805,  0.5189,  0.0858],
        [-1.1006,  0.6535,  0.3943,  ..., -1.6190, -1.2263,  0.0644],
        [-0.9457, -0.8053, -0.7376,  ..., -1.4371, -1.7079, -0.5195],
        ...,
        [ 0.0326,  0.0624, -0.9356,  ..., -1.6549, -1.0750,  0.1570],
        [-0.9285, -0.6192, -1.2086,  ..., -1.6032, -1.6032, -0.2125],
        [-1.1774,  2.9026,  2.7290,  ..., -0.7105, -1.5929, -0.1824]])
counter: 2, batch: tensor([[-0.9623, -0.0346,  0.1806,  ..., -0.5805,  0.5189,  0.0858],
     

In [34]:
for i, item in enumerate(ae_dataloader):
    print('Batch {}'.format(i))
    print(item)
    print("length: {}".format(len(item)))
    if i+1 >= 1:
        break

Batch 0
tensor([[-0.9623, -0.0346,  0.1806,  ..., -0.5805,  0.5189,  0.0858],
        [-1.1006,  0.6535,  0.3943,  ..., -1.6190, -1.2263,  0.0644],
        [-0.9457, -0.8053, -0.7376,  ..., -1.4371, -1.7079, -0.5195],
        ...,
        [ 0.0326,  0.0624, -0.9356,  ..., -1.6549, -1.0750,  0.1570],
        [-0.9285, -0.6192, -1.2086,  ..., -1.6032, -1.6032, -0.2125],
        [-1.1774,  2.9026,  2.7290,  ..., -0.7105, -1.5929, -0.1824]])
length: 128


In [53]:
# tensorboard logging
log_dir = "" + datetime.now().strftime("%Y%m%d-%H%M%S")
additional_comment = "just for checking what happens if"
writer = SummaryWriter(f"logs/autoencoder/{log_dir}", comment=f"Architecture=({input_dim}-{hidden_one_dim}-{hidden_two_dim}-{z_dim}), learning_rate={learning_rate}, beta={beta}, batch_size={batch_size}, number epochs={num_epochs}" + additional_comment)

# training loop
for epoch in range(num_epochs):
    
    ae.train()
        
    for train_iteration, batch in enumerate(tqdm(ae_dataloader, desc="Training epoch {}".format(epoch+1))):
        batch = batch.to(device)
        x_reconstructed = ae(batch)
        loss = loss_mse(batch, x_reconstructed)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        writer.add_scalar("ae train loss", loss.item(), epoch * len(ae_dataloader) + train_iteration)
        score = r2_score(x_reconstructed, batch).item()
        writer.add_scalar("ae R2 Score", score, epoch * len(ae_dataloader) + train_iteration)
    
    scheduler.step()

    ae.eval()
    with torch.no_grad():
        for val_iteration, batch in enumerate(tqdm(ae_val_dataloader, desc="Validation epoch {}".format(epoch+1))):
            batch = batch.to(device)

            x_reconstructed = ae(batch)

            score = r2_score(x_reconstructed, batch).item()

            writer.add_scalar("ae validation R2Score", score, epoch * len(ae_val_dataloader) + val_iteration)

    
        
        

Training epoch 1:   0%|          | 0/15 [00:00<?, ?it/s]/home/fes/Nextcloud/Uni/B.Sc. Bioinfo/Bachelorarbeit/thesis/src/dataloaders_and_sets/simple_dataset.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.tensor(self.data.iloc[index], dtype=torch.float32)
Validation epoch 1: 100%|██████████| 4/4 [00:00<00:00,  4.15it/s]


0.6205892821153005


Validation epoch 2: 100%|██████████| 4/4 [00:00<00:00,  4.11it/s]


0.3867015858491262


Validation epoch 3: 100%|██████████| 4/4 [00:01<00:00,  3.88it/s]


...

0.09284408340851466


Validation epoch 199: 100%|██████████| 4/4 [00:01<00:00,  3.68it/s]


0.09279358834028244


Validation epoch 200: 100%|██████████| 4/4 [00:01<00:00,  3.81it/s]

0.09274037033319474
